In [0]:
from pyspark.sql.functions import *
import urllib

In [0]:
display(dbutils.fs.ls("/FileStore/tables/amazon_databricks_project_accessKeys.csv"))

path,name,size,modificationTime
dbfs:/FileStore/tables/amazon_databricks_project_accessKeys.csv,amazon_databricks_project_accessKeys.csv,99,1691894678000


In [0]:
header = "true"
inferSchema = "true"
seperator = ","
aws_keys_df = spark.read.format("csv").option("header", header).option ("inferSchema", inferSchema).option("sep", seperator).load("/FileStore/tables/amazon_databricks_project_accessKeys.csv")

In [0]:
ACCESS_KEY = aws_keys_df.select("Access key ID").collect()[0]["Access key ID"]
SECRET_KEY = aws_keys_df.select("Secret access key").collect()[0]["Secret access key"]

ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
AWS_S3_BUCKET = "tokyo-olympic-project/raw-data"
MOUNT_NAME = "/mnt/tokyo-olympic-project"
SOURCE_URL = "s3a://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-214178352165429>:5
      2 MOUNT_NAME = "/mnt/tokyo-olympic-project"
      3 SOURCE_URL = "s3a://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
----> 5 dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o421.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyo-olympic-project; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyo-olympic-project
	at com.databricks.backend.daemon.data.client.DbfsCli

In [0]:
display(dbutils.fs.ls(MOUNT_NAME))

path,name,size,modificationTime
dbfs:/mnt/tokyo-olympic-project/Athletes.csv,Athletes.csv,418492,1692076926000
dbfs:/mnt/tokyo-olympic-project/Coaches.csv,Coaches.csv,16889,1692076927000
dbfs:/mnt/tokyo-olympic-project/EntriesGender.csv,EntriesGender.csv,1123,1692076928000
dbfs:/mnt/tokyo-olympic-project/Medals.csv,Medals.csv,2414,1692076929000
dbfs:/mnt/tokyo-olympic-project/Teams.csv,Teams.csv,35270,1692076930000
dbfs:/mnt/tokyo-olympic-project/transformed-data/,transformed-data/,0,1692165649677


In [0]:
# List directory contents
directory_contents = dbutils.fs.ls(MOUNT_NAME)

# Extract and print only the name column
file_names = [item.name for item in directory_contents]
for name in file_names:
    print("File Name:", name)

File Name: Athletes.csv
File Name: Coaches.csv
File Name: EntriesGender.csv
File Name: Medals.csv
File Name: Teams.csv


In [0]:
FILE_LOCATION = "/mnt/tokyo-olympic-project/"

for num, filename in enumerate(file_names, start=1):
    name = filename.replace(".csv", "").lower()
    file_path = f"{FILE_LOCATION}{filename}"

    # Create a DataFrame with an incremented name
    df_name = f"{name}_df"

    globals()[df_name] = spark.read.format("csv").option("header", header).option("sep", seperator).option("inferSchema", inferSchema).load(file_path)


In [0]:
# Find the top countries with the highest number of gold medals
top_10_gold_medal_countries = medals_df.orderBy("Gold", ascending=False).select("Rank", "Team_Country", "Gold").limit(10).show()

+----+--------------------+----+
|Rank|        Team_Country|Gold|
+----+--------------------+----+
|   1|United States of ...|  39|
|   2|People's Republic...|  38|
|   3|               Japan|  27|
|   4|       Great Britain|  22|
|   5|                 ROC|  20|
|   6|           Australia|  17|
|   7|         Netherlands|  10|
|   8|              France|  10|
|   9|             Germany|  10|
|  10|               Italy|  10|
+----+--------------------+----+



In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender_df.withColumn("Avg_Female", round(entriesgender_df["Female"] / entriesgender_df["Total"],2)
                            ).withColumn("Avg_Male", round(entriesgender_df["Male"] / entriesgender_df["Total"], 2))

average_entries_by_gender.show()


+--------------------+------+----+-----+----------+--------+
|          Discipline|Female|Male|Total|Avg_Female|Avg_Male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       1.0|     0.0|
|           Athletics|   969|1072| 2041|      0.47|    0.53|
|           Badminton|    86|  87|  173|       0.5|     0.5|
|   Baseball/Softball|    90| 144|  234|      0.38|    0.62|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|      0.35|    0.65|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|      0.49|    0.51|
|Cycling BMX Frees...|    10|   9|   19|      0.53|    0.47|
|  Cycling BMX Racing|  

In [0]:
TRANSFORMED_FILE_LOCATION = "/tokyo-olympic-project/transformed-data/"
athletes_df.write.mode("overwrite").option("header", "true").csv(f"{TRANSFORMED_FILE_LOCATION}/athletes")
coaches_df.write.mode("overwrite").option("header", "true").csv(f"{TRANSFORMED_FILE_LOCATION}/coaches")
entriesgender_df.write.mode("overwrite").option("header", "true").csv(f"{TRANSFORMED_FILE_LOCATION}/entriesgender")
medals_df.write.mode("overwrite").option("header", "true").csv(f"{TRANSFORMED_FILE_LOCATION}/medals")
teams_df.write.mode("overwrite").option("header", "true").csv(f"{TRANSFORMED_FILE_LOCATION}/teams")

In [0]:
athletes_df.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
dbutils.unmount(MOUNT_NAME)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-214178352165426>:1
----> 1 dbutils.unmount(MOUNT_NAME)

File /databricks/python_shell/dbruntime/dbutils.py:570, in DBUtils.__getattr__(self, item)
    567     self.data = DataHandler(self.entry_point, self.displayHTML, self.sqlContext)
    568     return self.data
--> 570 raise AttributeError

AttributeError: 